# OD Data Preparation
this notebook takes the preprocessed OD data and turns it into csvs to be used for visualization

##### Preprocessed Data
The preprocessed origin-destination data consists of 168 separate csv-files, one for each hour of the week.
Each file contains two rows for every trip registered within the given hour and day, one with data on the origin of the trip and one with data on the destination.
The columns provide a unix-timestamp, longitude and latitude, the travelled distance in kilometres, the travelled time in seconds, the trip-id, the number of cells that are travelled through, whether the row represents the start or the end of a trip and the modelled mode in which the majority of trip segments are taken.

##### Combining the Individual Files, Handling Timestamps and Geometry
The csv-files containing the preprocessed data are loaded as *pandas DataFrames* and aggregated one by one as to prevent a lack of working memory since the files containing individual trips are rather big.
The individual *DataFrames* are split into two *DataFrames* containing the origin-data and the destination-data respectively and merged again, so that each trip is represented by one row only containing both the data on the origin of the trip as well as the destination.
Columns containing *shapely Points* with the origin and destination coordinates are added and two *GeoDataFrames* are extracted, using the *geopandas* package, one containing the points of origin as its geometry and one containing the points of destination as its geometry.
Both *GeoDataFrames* are then spatially joined with a third *GeoDataFrame* containing the network cell polygon geometries saved in *celldf.csv* and the cell-IDs for the cells of origin and destination are added to the *GeoDataFrames*.
The two *GeoDataFrames* are then again merged to one *DataFrame*.
The timestamps are translated and extended analogous to the ones in the flow data (see *D_1+2_flows_DataPrep.ipynb*) and the *DataFrame* is then aggregated to movements between different cells within one hour of the week.
Travelled distances, times and the number of passed cells are averaged while the movements within each mode are counted.
The entries in the column *mode_prediction_majority* are simplified.
Finally, each *DataFrame* is appended to a single *DataFrame* resulting in one *DataFrame* containing all OD data.
Further columns for the weekday, the day-type and a list of coordinates connecting the center point of the cell of origin to the center point of the destination cell are added and it is saved as **all.csv**.

##### Preparing for Flow Maps (Map Vis) and Temporal Overviews (Cycle Vis)
From the *DataFrame* containing all movements, a *pivot_table* is created, merging all rows containing data on the same spatial, temporal and directional entities into one row, thus adding columns for the mode-specific data.
Movement counts are summed over while data on the travelled distance, time and number of network cells is averaged for each mode and spatio-temporal relation.
Finally, columns for the summed public modes and the sum of movements of all modes are added and the *DataFrame* is saved as **cycleAndFlowODdf.csv**.

##### Computing Anomalies
For the visualization of anomalies, a *DataFrame* is constructed analogous to that of flow data anomalies (see *D_1+2_flows_DataPrep.ipynb*), containing averaged movement counts of Monday, Tuesday and Thursday, the data of Wednesday and the respective anomalies. The *DataFrame* is saved as **anomODdf.csv**.

##### Versions of the used packages:
- pandas: 0.24.2
- numpy: 1.16.4
- shapely: 1.6.4.post1
- geopandas: 0.6.1

In [1]:
import pandas as pd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import Point, LineString, Polygon
import geopandas as gpd

## load preprocessed data
merge start and end rows and sjoin with polygon cells from celldf

done individually because of lack of memory capacity

In [2]:
def ODcsvtodf(day, hour):
    data = pd.read_csv('data/OD_with_mode/od_munich_day_'+str(day)+'_month_3_hour_'+str(hour)+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)    
    df = pd.DataFrame(data)
    df.columns = ['time', 'avg_lon', 'avg_lat', 'dist', 'traveltime', 'trip_id', 'cellsN', 'point_type', 'mode_prediction_majority']
    return df;

def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

In [3]:
celldf = csvtodf_SC('flows_with_mode/aggregations/celldf')
celldf['geometry'] = celldf['geometry'].apply(loads)
celldf = celldf.reindex(columns = ['cellID', 'centroidCoords', 'geometry'])
cellgdf = gpd.GeoDataFrame(celldf, geometry='geometry')
cellgdf.head()

,cellID,centroidCoords,geometry
0,1,"[48.15396752884293, 11.51032914199828]","POLYGON ((11.51079 48.14606, 11.51421 48.14625..."
1,2,"[48.22263647015427, 11.62774480807693]","POLYGON ((11.63551 48.23591, 11.63390 48.23707..."
2,3,"[48.13564057367491, 11.70093894926043]","POLYGON ((11.69622 48.14901, 11.69910 48.15361..."
3,4,"[48.13194111905815, 11.54797042679206]","POLYGON ((11.54156 48.12880, 11.55160 48.12528..."
4,5,"[48.19669108307331, 11.6113147037113]","POLYGON ((11.61582 48.18833, 11.60391 48.19367..."


In [4]:
def moveMidnight(row):
    if (row.hour < 2):
        if (row.dayInt == 0):
            return 6;
        else:
            return row.dayInt-1
    else:
        return row.dayInt;

Modes = ['privat', 'Rail', 'UBahn', 'Tram', 'Bus']
newModes = {#'privat':'privat',
            #'Rail':'Rail',
            'Mode::Subway':'UBahn',
            'Mode::Tram':'Tram',
            'Mode::Bus':'Bus'}

## merge all csvs into one
sjoining with the cell polygons and aggregating over hours of each day for each mode

In [5]:
allODdf = pd.DataFrame()
for i in range(11,18):
    for j in range(0,24):
        df = ODcsvtodf(i, j)
        
        startdf = df[df.point_type == 'start'].copy()
        enddf = df[df.point_type == 'end'].copy()
        
        df = pd.merge(startdf, enddf,  how='outer', on=['trip_id'])
        df.columns = ['time', 'startlon', 'startlat', 'dist', 'traveltime', 'trip_id',
                      'cellsN', 'point_type_x', 'mode_prediction_majority', 'time_y',
                      'endlon', 'endlat', 'dist_y', 'traveltime_y', 'cellsN_y',
                      'point_type_y', 'mode_prediction_majority_y']
        df = df.reindex(columns = ['time', 'traveltime',
                                   'startlon', 'startlat', 'endlon', 'endlat',
                                   'dist', 'cellsN', 'mode_prediction_majority'])
        df['startPoint'] = [Point(xy) for xy in zip(df.startlon, df.startlat)]
        df['endPoint'] = [Point(xy) for xy in zip(df.endlon, df.endlat)]
        
        gdf = gpd.GeoDataFrame(df, geometry = 'startPoint')
        startdf = gpd.sjoin(gdf, cellgdf, op='within')
        startdf = startdf.reindex(columns = ['time', 'traveltime', 'startlon', 'startlat', 'endlon', 'endlat',
                                             'dist', 'cellsN', 'mode_prediction_majority', 'startPoint', 'endPoint',
                                             'cellID', 'centroidCoords'])
        startdf.columns = ['time', 'traveltime', 'startlon', 'startlat', 'endlon', 'endlat',
                                             'dist', 'cellsN', 'mode_prediction_majority', 'startPoint', 'endPoint',
                                             'startCellID', 'startCentroidCoords']
        
        gdf = gpd.GeoDataFrame(df, geometry = 'endPoint')
        enddf = gpd.sjoin(gdf, cellgdf, op='within')
        enddf = enddf.reindex(columns = ['time', 'traveltime', 'startlon', 'startlat', 'endlon', 'endlat',
                                         'dist', 'cellsN', 'mode_prediction_majority', 'startPoint', 'endPoint',
                                         'cellID', 'centroidCoords'])
        enddf.columns = ['time', 'traveltime', 'startlon', 'startlat', 'endlon', 'endlat',
                         'dist', 'cellsN', 'mode_prediction_majority', 'startPoint', 'endPoint',
                         'endCellID', 'endCentroidCoords']
        
        df = pd.merge(startdf, enddf,  how='outer', on=['time', 'traveltime', 'startlon', 'startlat', 'endlon', 'endlat',
                           'dist', 'cellsN', 'mode_prediction_majority'])
        df = df.reindex(columns = ['time', 'traveltime', 'dist', 'cellsN', 'mode_prediction_majority',
                                   'startCellID', 'endCellID', 'startCentroidCoords', 'endCentroidCoords'])
        
        df['dateTime'] = pd.to_datetime(df.time, unit='s')
        df['dayInt'] = df.dateTime.dt.dayofweek
        df['hour'] = df.dateTime.dt.hour
        df['dayInt'] = df.apply(lambda row: moveMidnight(row), axis=1)
        
        df = df.groupby(['dayInt', 'hour', 'startCellID', 'endCellID',
                         'mode_prediction_majority']).agg({'startCentroidCoords':['first'],
                                                           'endCentroidCoords':['first'],
                                                           'dist':['mean'],
                                                           'cellsN':['mean'],
                                                           'traveltime':['mean'],
                                                           'time':['count']}).reset_index()
        df.columns = df.columns.get_level_values(0)
        df.columns = ['dayInt', 'hour', 'startCellID', 'endCellID', 'mode_prediction_majority',
                      'startCentroidCoords', 'endCentroidCoords', 'dist', 'cellsN', 'traveltime', 'N']
        df = df.replace({'mode_prediction_majority': newModes})
        
        allODdf = allODdf.append(df, ignore_index=True, sort=False)

In [6]:
allODdf.head()

,dayInt,hour,startCellID,endCellID,mode_prediction_majority,startCentroidCoords,endCentroidCoords,dist,cellsN,traveltime,N
0,6,0,1,1,Bus,"[48.15396752884293, 11.51032914199828]","[48.15396752884293, 11.51032914199828]",0.775000,2.000000,45.000000,1
1,6,0,1,1,privat,"[48.15396752884293, 11.51032914199828]","[48.15396752884293, 11.51032914199828]",0.017583,3.250000,480.166667,12
2,6,0,1,23,privat,"[48.15396752884293, 11.51032914199828]","[48.16477666664996, 11.50581476144185]",0.430333,6.666667,1040.666667,3
3,6,0,1,34,privat,"[48.15396752884293, 11.51032914199828]","[48.1773186792974, 11.57529006886845]",7.613000,5.000000,1273.000000,1
4,6,0,1,36,Bus,"[48.15396752884293, 11.51032914199828]","[48.13610079361809, 11.53260950394832]",3.140000,3.000000,222.000000,1


In [8]:
allODdf.to_csv('data/OD_with_mode/all.csv', index=False, sep=';')

## enhance csv:
add weekday, daytype and ODCoords

In [120]:
allODdf = csvtodf_SC('OD_with_mode/all')

In [121]:
def getWeekday(row):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return days[row.dayInt];

def getDayType(row):
    dayTypes = ['MonThu', 'MonThu', 'MonThu', 'MonThu', 'Fri', 'Weekend', 'Weekend']
    return dayTypes[row.dayInt];

def getLineCoords(row):
    return [row.startCentroidCoords, row.endCentroidCoords];

allODdf['ODCoords'] = allODdf.apply(lambda row: getLineCoords(row), axis=1)
allODdf['weekday'] = allODdf.apply(lambda row: getWeekday(row), axis=1)
allODdf['dayType'] = allODdf.apply(lambda row: getDayType(row), axis=1)

In [122]:
allODdf  = allODdf.reindex(columns = ['startCellID', 'endCellID', 'ODCoords',
                                      'dayInt', 'hour', 'weekday', 'dayType',
                                      'mode_prediction_majority', 'N',
                                     'dist', 'cellsN', 'traveltime'])

In [125]:
allODdf.head()

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,mode_prediction_majority,N,dist,cellsN,traveltime
0,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",6,0,Sunday,Weekend,Bus,1,0.775000,2.000000,45.000000
1,1,1,"[[48.15396752884293, 11.51032914199828], [48.1...",6,0,Sunday,Weekend,privat,12,0.017583,3.250000,480.166667
2,1,23,"[[48.15396752884293, 11.51032914199828], [48.1...",6,0,Sunday,Weekend,privat,3,0.430333,6.666667,1040.666667
3,1,34,"[[48.15396752884293, 11.51032914199828], [48.1...",6,0,Sunday,Weekend,privat,1,7.613000,5.000000,1273.000000
4,1,36,"[[48.15396752884293, 11.51032914199828], [48.1...",6,0,Sunday,Weekend,Bus,1,3.140000,3.000000,222.000000


In [124]:
allODdf.to_csv('data/OD_with_mode/allODdf.csv', index=False, sep=';')

## aggregate for mapVis and cycleVis
sum N and aggregate mean of dist, cellsN and traveltime for each mode

In [10]:
cycleODdf = csvtodf_SC('OD_with_mode/allODdf')

In [11]:
cycleODdf.head()

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,mode_prediction_majority,N,dist,cellsN,traveltime
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",6,0,Sunday,Weekend,Bus,1,0.775000,2.000000,45.000000
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",6,0,Sunday,Weekend,privat,12,0.017583,3.250000,480.166667
2,1,23,"['[48.15396752884293, 11.51032914199828]', '[4...",6,0,Sunday,Weekend,privat,3,0.430333,6.666667,1040.666667
3,1,34,"['[48.15396752884293, 11.51032914199828]', '[4...",6,0,Sunday,Weekend,privat,1,7.613000,5.000000,1273.000000
4,1,36,"['[48.15396752884293, 11.51032914199828]', '[4...",6,0,Sunday,Weekend,Bus,1,3.140000,3.000000,222.000000


In [12]:
cycleODdf = cycleODdf.pivot_table(values = ['N', 'dist', 'cellsN', 'traveltime'],
                                  index=['startCellID', 'endCellID', 'ODCoords', 'dayInt', 'hour', 'weekday', 'dayType'],
                                  columns='mode_prediction_majority',
                                  aggfunc={'N':['sum'],
                                           'dist':['mean'],
                                           'cellsN':['mean'],
                                           'traveltime':['mean']}).copy().fillna(0).reset_index()
cycleODdf.columns = cycleODdf.columns.get_level_values(0) +'_'+ cycleODdf.columns.get_level_values(1) +'_'+ cycleODdf.columns.get_level_values(2)
cycleODdf.columns = ['startCellID', 'endCellID', 'ODCoords', 'dayInt', 'hour', 'weekday', 'dayType',
                     'N_sum_Bus', 'N_sum_Rail', 'N_sum_Tram', 'N_sum_UBahn', 'N_sum_privat',
                     'cellsN_mean_Bus', 'cellsN_mean_Rail', 'cellsN_mean_Tram', 'cellsN_mean_UBahn', 'cellsN_mean_privat',
                     'dist_mean_Bus', 'dist_mean_Rail', 'dist_mean_Tram', 'dist_mean_UBahn', 'dist_mean_privat',
                     'traveltime_mean_Bus', 'traveltime_mean_Rail', 'traveltime_mean_Tram', 'traveltime_mean_UBahn', 'traveltime_mean_privat']
cycleODdf.columns.name = ''

In [13]:
cycleODdf.head()

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,N_sum_Bus,N_sum_Rail,N_sum_Tram,...,dist_mean_Bus,dist_mean_Rail,dist_mean_Tram,dist_mean_UBahn,dist_mean_privat,traveltime_mean_Bus,traveltime_mean_Rail,traveltime_mean_Tram,traveltime_mean_UBahn,traveltime_mean_privat
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,0,Monday,MonThu,2.0,0.0,0.0,...,0.263,0.0,0.0,0.0,0.067643,1005.0,0.0,0.0,0.0,656.071429
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,1,Monday,MonThu,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.116500,0.0,0.0,0.0,0.0,556.000000
2,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,2,Monday,MonThu,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.128065,0.0,0.0,0.0,0.0,440.225806
3,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,3,Monday,MonThu,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.276417,0.0,0.0,0.0,0.0,594.583333
4,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,4,Monday,MonThu,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.192296,0.0,0.0,0.0,0.0,598.962963


In [14]:
cycleODdf.to_csv('data/OD_with_mode/aggregations/cycleAndFlowODdf.csv', index=False, sep=';')

## add moves and public sums and means

In [7]:
from ast import literal_eval

In [8]:
def csvtodf_SC(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=';',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def csvtodf_C(path):
    data = pd.read_csv('data/'+path+'.csv',
                       delimiter=',',
                       skipinitialspace=True,
                       skiprows=0)
    df = pd.DataFrame(data)
    return df;

def getCoordsBack(coords):
    return literal_eval(coords);

In [3]:
modes = ['moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus']

traveltime_mean_modes = ['']*7
for i in range(0,7):
    traveltime_mean_modes[i] = 'traveltime_mean_'+modes[i]

dist_mean_modes = ['']*7
for i in range(0,7):
    dist_mean_modes[i] = 'dist_mean_'+modes[i]

cellsN_mean_modes = ['']*7
for i in range(0,7):
    cellsN_mean_modes[i] = 'cellsN_mean'+modes[i]

In [16]:
def sumPublic(row):
    return row.Bus + row.Rail + row.Tram + row.UBahn;
def sumMoves(row):
    return row.public + row.privat;

def meanPubTT(row):
    i=0
    means = [0]*4
    for tt_mode in traveltime_mean_modes[3:]:
        means[i] = row[tt_mode]
        i = i+1
    return np.mean(means);

def meanPubDist(row):
    i=0
    means = [0]*4
    for dist_mode in dist_mean_modes[3:]:
        means[i] = row[dist_mode]
        i = i+1
    return np.mean(means);

def meanMovesTT(row):
    return 0.5*(row.traveltime_mean_public + row.traveltime_mean_privat);

def meanMovesDist(row):
    return 0.5*(row.dist_mean_public + row.dist_mean_privat);

In [9]:
ODdf = csvtodf_SC('OD_with_mode/aggregations/cycleAndFlowODdf')
ODdf['ODCoords'] = ODdf.apply(lambda row: getCoordsBack(row.ODCoords), axis = 1)
ODdf.columns = ['startCellID', 'endCellID', 'ODCoords',
                'dayInt', 'hour', 'weekday', 'dayType',
                'Bus', 'Rail', 'Tram', 'UBahn', 'privat',
                'cellsN_mean_Bus', 'cellsN_mean_Rail', 'cellsN_mean_Tram', 'cellsN_mean_UBahn', 'cellsN_mean_privat',
                'dist_mean_Bus', 'dist_mean_Rail', 'dist_mean_Tram', 'dist_mean_UBahn', 'dist_mean_privat',
                'traveltime_mean_Bus', 'traveltime_mean_Rail', 'traveltime_mean_Tram', 'traveltime_mean_UBahn', 'traveltime_mean_privat']

In [10]:
ODdf['public'] = ODdf.apply(lambda row: sumPublic(row), axis=1)
ODdf['moves'] = ODdf.apply(lambda row: sumMoves(row), axis=1)

In [17]:
ODdf['traveltime_mean_public'] = ODdf.apply(lambda row: meanPubTT(row), axis=1)
ODdf['dist_mean_public'] = ODdf.apply(lambda row: meanPubDist(row), axis=1)

In [18]:
ODdf['traveltime_mean_moves'] = ODdf.apply(lambda row: meanMovesTT(row), axis=1)
ODdf['dist_mean_moves'] = ODdf.apply(lambda row: meanMovesDist(row), axis=1)

In [19]:
ODdf.to_csv('data/OD_with_mode/aggregations/cycleAndFlowODdf.csv', index=False, sep=';')

## df for Wednesday Anomalies

In [11]:
cycleODdf = csvtodf_SC('OD_with_mode/aggregations/cycleAndFlowODdf')
cycleODdf.head(2)

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,Bus,Rail,Tram,...,traveltime_mean_Rail,traveltime_mean_Tram,traveltime_mean_UBahn,traveltime_mean_privat,public,moves,traveltime_mean_public,dist_mean_public,traveltime_mean_moves,dist_mean_moves
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,0,Monday,MonThu,2.0,0.0,0.0,...,0.0,0.0,0.0,656.071429,2.0,16.0,251.25,0.06575,453.660714,0.066696
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,1,Monday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,556.000000,0.0,8.0,0.00,0.00000,278.000000,0.058250


In [9]:
modidoODdf = cycleODdf[cycleODdf.dayInt.isin([0,1,3])].copy()
modidoODdf = modidoODdf.groupby(['startCellID', 'endCellID', 'ODCoords', 'hour']).agg('mean').reset_index()
modidoODdf.columns = modidoODdf.columns.get_level_values(0)
modidoODdf.columns = ['startCellID', 'endCellID', 'ODCoords', 'hour', 'dayInt',
                      'Bus_MDD', 'Rail_MDD', 'Tram_MDD', 'UBahn_MDD', 'privat_MDD',
                      'cellsN_mean_Bus_MDD', 'cellsN_mean_Rail_MDD', 'cellsN_mean_Tram_MDD', 'cellsN_mean_UBahn_MDD', 'cellsN_mean_privat_MDD',
                      'dist_mean_Bus_MDD', 'dist_mean_Rail_MDD', 'dist_mean_Tram_MDD', 'dist_mean_UBahn_MDD', 'dist_mean_privat_MDD',
                      'traveltime_mean_Bus_MDD', 'traveltime_mean_Rail_MDD', 'traveltime_mean_Tram_MDD', 'traveltime_mean_UBahn_MDD', 'traveltime_mean_privat_MDD',
                      'public_MDD', 'moves_MDD', 'traveltime_mean_public_MDD', 'dist_mean_public_MDD', 'traveltime_mean_moves_MDD', 'dist_mean_moves_MDD']

In [12]:
modidoODdf.head(2)

,startCellID,endCellID,ODCoords,hour,dayInt,Bus_MDD,Rail_MDD,Tram_MDD,UBahn_MDD,privat_MDD,...,traveltime_mean_Rail_MDD,traveltime_mean_Tram_MDD,traveltime_mean_UBahn_MDD,traveltime_mean_privat_MDD,public_MDD,moves_MDD,traveltime_mean_public_MDD,dist_mean_public_MDD,traveltime_mean_moves_MDD,dist_mean_moves_MDD
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,1.333333,0.666667,0.0,0.0,0.0,13.666667,...,0.0,0.0,0.0,427.707143,0.666667,14.333333,83.750000,0.021917,255.728571,0.061063
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",1,1.333333,0.666667,0.0,0.0,0.0,9.333333,...,0.0,0.0,0.0,503.300000,0.666667,10.000000,93.708333,0.000000,298.504167,0.073083


In [16]:
miODdf = cycleODdf[cycleODdf.dayInt == 2].copy()
miODdf.head(2)

,startCellID,endCellID,ODCoords,dayInt,hour,weekday,dayType,Bus,Rail,Tram,...,traveltime_mean_Rail,traveltime_mean_Tram,traveltime_mean_UBahn,traveltime_mean_privat,public,moves,traveltime_mean_public,dist_mean_public,traveltime_mean_moves,dist_mean_moves
48,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",2,0,Wednesday,MonThu,0.0,0.0,0.0,...,0.0,0.0,0.0,710.153846,0.0,13.0,0.0,0.00000,355.076923,0.041962
49,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",2,1,Wednesday,MonThu,1.0,0.0,0.0,...,0.0,0.0,0.0,188.000000,1.0,3.0,91.0,0.13625,139.500000,0.329125


In [17]:
anomODdf = pd.merge(miODdf, modidoODdf, how='outer', on=['startCellID', 'endCellID', 'hour'])
anomODdf['ODCoords_x'] = anomODdf['ODCoords_x'].fillna(anomODdf['ODCoords_y'])
anomODdf = anomODdf.reindex(columns = ['startCellID', 'endCellID', 'ODCoords_x', 'hour',
                                       'moves', 'privat', 'public', 'Rail', 'UBahn', 'Tram', 'Bus',
                                       'moves_MDD', 'privat_MDD', 'public_MDD', 'Rail_MDD', 'UBahn_MDD', 'Tram_MDD', 'Bus_MDD',
                                       'dist_mean_moves', 'dist_mean_privat', 'dist_mean_public', 'dist_mean_Rail', 'dist_mean_UBahn', 'dist_mean_Tram', 'dist_mean_Bus',
                                       'traveltime_mean_moves', 'traveltime_mean_privat', 'traveltime_mean_public','traveltime_mean_Rail', 'traveltime_mean_UBahn', 'traveltime_mean_Tram', 'traveltime_mean_Bus'])
anomODdf = anomODdf.fillna(0)

In [34]:
anomODdf.columns = ['startCellID',
 'endCellID',
 'ODCoords',
 'hour',
 'moves',
 'privat',
 'public',
 'Rail',
 'UBahn',
 'Tram',
 'Bus',
 'moves_MDD',
 'privat_MDD',
 'public_MDD',
 'Rail_MDD',
 'UBahn_MDD',
 'Tram_MDD',
 'Bus_MDD',
 'dist_mean_moves',
 'dist_mean_privat',
 'dist_mean_public',
 'dist_mean_Rail',
 'dist_mean_UBahn',
 'dist_mean_Tram',
 'dist_mean_Bus',
 'traveltime_mean_moves',
 'traveltime_mean_privat',
 'traveltime_mean_public',
 'traveltime_mean_Rail',
 'traveltime_mean_UBahn',
 'traveltime_mean_Tram',
 'traveltime_mean_Bus']

In [35]:
anomODdf.head()

,startCellID,endCellID,ODCoords,hour,moves,privat,public,Rail,UBahn,Tram,...,dist_mean_UBahn,dist_mean_Tram,dist_mean_Bus,traveltime_mean_moves,traveltime_mean_privat,traveltime_mean_public,traveltime_mean_Rail,traveltime_mean_UBahn,traveltime_mean_Tram,traveltime_mean_Bus
0,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",0,13.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,355.076923,710.153846,0.00,0.0,0.0,0.0,0.0
1,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",1,3.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.545,139.500000,188.000000,91.00,0.0,0.0,0.0,364.0
2,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",2,14.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,221.000000,442.000000,0.00,0.0,0.0,0.0,0.0
3,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",3,7.0,6.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.727,507.208333,923.666667,90.75,0.0,0.0,0.0,363.0
4,1,1,"['[48.15396752884293, 11.51032914199828]', '[4...",4,19.0,17.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.263,431.073529,519.647059,342.50,1112.0,0.0,0.0,258.0


In [5]:
anomODdf['movesAnom'] = anomODdf.moves - anomODdf.moves_MDD
anomODdf['privatAnom'] = anomODdf.privat - anomODdf.privat_MDD
anomODdf['publicAnom'] = anomODdf.public - anomODdf.public_MDD
anomODdf['RailAnom'] = anomODdf.Rail - anomODdf.Rail_MDD
anomODdf['UBahnAnom'] = anomODdf.UBahn - anomODdf.UBahn_MDD
anomODdf['TramAnom'] = anomODdf.Tram - anomODdf.Tram_MDD
anomODdf['BusAnom'] = anomODdf.Bus - anomODdf.Bus_MDD

In [6]:
anomODdf.to_csv('data/OD_with_mode/aggregations/anomODdf.csv', index=False, sep=';')